# <span style="color:purple">Resilient Code</span>

## <span style="color:purple">Ph.D. Workshop - Session 3</span>


## July 15, 2022

# <span style="color:purple">What is code resilience?</span>
<br>

<span style="color:purple">Code that is capable of performing without failure (or at least failing gracefully) under a wide range of conditions. Code must be able to:
</span>
<br>

* handle unexpected inputs

* handle unexpected events

* make it possible to add new features

* make it possible to fix bugs

# <span style="color:purple">Topics</span>
<br>

1. Creating Environments
2. Writing Clean Code
3. Handling Errors and Exceptions
4. Writing Tests

# <span style="color:purple">1. Creating Environments</span>
<br>


# <span style="color:purple">Reproducibility</span>
<br>
<center><img src="figures/reproducibility.png" width="60%" style='border:5px solid #000000'/></center>

# <span style="color:purple">2: Writing Clean Code</span>

<span style="color:darkblue">There are two ways of constructing a software design: One way is to make it so simple that there are obviously no deficiencies, and the other way is to make it so complicated that there are no obvious deficiencies. The first method is far more difficult. (C.A.R. Hoare)</span>

# <span style="color:purple">Don't Repeat Yourself!</span>
<br>

<table><tr>
<td> <img src="figures/keep-dry.jpg" style="width: 500px;"/> </td>
</tr></table>



In [ ]:
from pathlib import Path
from rich import print

file = Path("./data/0001012975-17-000759.txt")
print(file.read_text())

In [ ]:
from edgar.utils import create_doc

doc = create_doc(file)
print(doc.doc_info)
print(doc.report_owners)

In [ ]:
import csv

out_dir = Path("./temp")

doc = create_doc(file)
out_file=out_dir / "document_info.csv"
row_dicts = [doc.doc_info]
if len(row_dicts) == 0:
    pass
if not out_file.exists():
    csv_writer = csv.writer(open(out_file, "a+", newline=""))
    csv_writer.writerow(row_dicts[0].keys())
with open(out_file, "a+", newline="") as f:
    csv_writer = csv.writer(f, quoting=csv.QUOTE_ALL)
    for row_dict in row_dicts:
        csv_writer.writerow(row_dict.values())
        
out_file=out_dir / "report_owners.csv"
row_dicts = doc.report_owners
if len(row_dicts) == 0:
    pass
if not out_file.exists():
    csv_writer = csv.writer(open(out_file, "a+", newline=""))
    csv_writer.writerow(row_dicts[0].keys())
with open(out_file, "a+", newline="") as f:
    csv_writer = csv.writer(f, quoting=csv.QUOTE_ALL)
    for row_dict in row_dicts:
        csv_writer.writerow(row_dict.values())

In [1]:
from edgar.utils import write_records

??write_records

In [ ]:
out_dir = Path("./temp")

write_records(
    [doc.doc_info], 
    out_file=out_dir/"document_info.csv"
)
write_records(
    doc.report_owners, 
    out_file=out_dir/"report_owners.csv"
)

# <span style="color:purple">Prefer to use Pre-Existing Libraries</span>
<br>
<center><img src="figures/functional.webp" width="75%" style='border:3px solid #000000'/></center>

In [2]:
import numpy as np

array = np.random.rand(200_000)
print(len(array))
print(array)

200000
[0.32542385 0.83287718 0.864878   ... 0.67584775 0.78238292 0.68153343]


In [3]:
%%time
total = 0
for i in range(len(array)):
    total += array[i]
total

CPU times: user 42.8 ms, sys: 2.26 ms, total: 45.1 ms
Wall time: 44.3 ms


99837.97045458334

In [4]:
%%time
np.sum(array)

CPU times: user 347 µs, sys: 64 µs, total: 411 µs
Wall time: 232 µs


99837.97045458162

# <span style="color:purple">Scripts, Modules, or Notebooks?</span>
<br>

<center><img src="figures/notes.jpg" width="50%" style='border:5px solid #000000'/></center>

# <span style="color:purple">3: Handling Errors and Exceptions</span>
<br>
<br>
<center><img src="figures/castle.jpg" width="50%" style='border:5px solid #000000'/></center>

# <span style="color:purple">Try-Catch Blocks</span>
<br>

In [ ]:
numbers = list(range(10))
print(numbers)
print(numbers[5])
print(numbers[11])

In [ ]:
def raise_an_exception(flag):
    if flag:
        raise Exception("oops!")
    else:
        print("success!")
        
raise_an_exception(False)
raise_an_exception(True)
print("hello world!")

In [ ]:
try:
    raise_an_exception(True)
except Exception as e:
    print(e)
print("hello world!")

# <span style="color:purple">Example: Bonusing MTurk Workers</span>


```python
# original version
def send_bonus(client: BaseClient, df: pd.DataFrame) -> pd.DataFrame:
    rows = df.to_dict(orient="records")
    for row in rows:
        mid = row["mid"]
        bonus = row["bonus"]
        message = row["message"]
        assignment_id = row["assignment_id"]
        response = client.send_bonus(
            WorkerId=mid,
            BonusAmount=f"{bonus}",
            AssignmentId=assignment_id,
            Reason=message,
        row["response"] = response
    return pd.DataFrame(rows)

            
# calling code
result_df = m.send_bonus(client, bonus_data, force)
result_df.to_csv(
    out_dir / f"bonuses_{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.csv",
    index=False,
)
```

```python
# updated version
def send_bonus(client: BaseClient, df: pd.DataFrame, force: bool) -> pd.DataFrame:
    rows = df.to_dict(orient="records")
    for row in rows:
        mid = row["mid"]
        bonus = row["bonus"]
        message = row["message"]
        assignment_id = row["assignment_id"]
        try:
            if bonus == 0:
                raise Exception(
                    f"Trying to send $0.00 bonus for assignment {assignment_id}"
                )
            prev_bonuses = list_bonus_payments_for_assignment(client, assignment_id)
            if len(prev_bonuses) > 0 and not force:
                raise Exception(
                    f"Previous bonus has been sent for assignment {assignment_id}"
                )
            response = client.send_bonus(
                WorkerId=mid,
                BonusAmount=f"{bonus}",
                AssignmentId=assignment_id,
                Reason=message,
            )
            row["response"] = response
        except Exception as e:
            print(e)
            row["response"] = str(e)
    return pd.DataFrame(rows)

            
# calling code
result_df = m.send_bonus(client, bonus_data, force)
result_df.to_csv(
    out_dir / f"bonuses_{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.csv",
    index=False,
)
```

# <span style="color:purple">4: Writing Tests</span>
<center><img src="figures/bug.jpg" width="40%" style='border:5px solid #000000'/></center>
<br>

<span style="color:darkblue">“More than the act of testing, the act of designing tests is one of the best bug preventers known. The thinking that must be done to create a useful test can discover and eliminate bugs before they are coded – indeed, test-design thinking can discover and eliminate bugs at every stage in the creation of software, from conception to specification, to design, coding and the rest.” (Boris Beizer)</span>

# <span style="color:purple">Agile Coding</span>
<br>
<center><img src="figures/agile.jpg" width="60%" style='border:5px solid #000000'/></center>
<br>

# <span style="color:purple">Types of Testing</span>
<br>
<center><img src="figures/testing-levels.png" width="60%" style='border:5px solid #000000'/></center>
<br>

# <span style="color:purple">Unit Testing</span>
<br>
<center><img src="figures/unit-testing-puzzle.png" width="50%" style='border:5px solid #000000'/></center>
<br>
* Goal: test a single “unit” of the code in isolation.  For instance, execute a single function with a range of inputs and validate the corresponding outputs

# <span style="color:purple">Unit Testing</span>
<br>

* Proper unit testing is automatic, not manual
* Each unit test has three parts: (1) setup input (2) activate code, and (3) assert result
* Best to write code and tests at the same time!